In [5]:
!pip install pyspark

In [7]:
# Data manipulation and Graphing
import numpy as np # Linear algebra
import pandas as pd #data processing, CSV file I/O
import os # Interact with OS file system
import seaborn as sns # Making graphs and visualising data

# Basic Setup
from pyspark import SparkConf, SparkContext #
from pyspark.sql import SparkSession, SQLContext # Working with DataFrames

# Spark Tools
from pyspark.sql.types import * # Schema tools for DataFrames
import pyspark.sql.functions as F # 
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression # Linear regression model
from pyspark.mllib.evaluation import RegressionMetrics # Model evaluation metrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel # Hyperparameter tuning
from pyspark.ml.feature import VectorAssembler, StandardScaler # Prepare features for machine learning
from pyspark.ml.evaluation import RegressionEvaluator # Evaluation

In [9]:
import os
# Find data set in file system
for dirname, _, filenames in os.walk(r'C:\Users\washi\Downloads\archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Users\washi\Downloads\archive\AmesHousing.csv


In [11]:
from pyspark.sql import SparkSession

In [13]:
# Create a spark session
spark = SparkSession.builder.appName('Dataframe').getOrCreate()

In [14]:
# Read over dataset
df_housing = spark.read.option('header', 'true').csv(os.path.join(dirname, 'AmesHousing.csv'), inferSchema = True)

In [19]:
# Show the first 3 rows
df_housing.head(3)

[Row(Order=1, PID=526301100, MS SubClass=20, MS Zoning='RL', Lot Frontage=141, Lot Area=31770, Street='Pave', Alley='NA', Lot Shape='IR1', Land Contour='Lvl', Utilities='AllPub', Lot Config='Corner', Land Slope='Gtl', Neighborhood='NAmes', Condition 1='Norm', Condition 2='Norm', Bldg Type='1Fam', House Style='1Story', Overall Qual=6, Overall Cond=5, Year Built=1960, Year Remod/Add=1960, Roof Style='Hip', Roof Matl='CompShg', Exterior 1st='BrkFace', Exterior 2nd='Plywood', Mas Vnr Type='Stone', Mas Vnr Area=112, Exter Qual='TA', Exter Cond='TA', Foundation='CBlock', Bsmt Qual='TA', Bsmt Cond='Gd', Bsmt Exposure='Gd', BsmtFin Type 1='BLQ', BsmtFin SF 1=639, BsmtFin Type 2='Unf', BsmtFin SF 2=0, Bsmt Unf SF=441, Total Bsmt SF=1080, Heating='GasA', Heating QC='Fa', Central Air='Y', Electrical='SBrkr', 1st Flr SF=1656, 2nd Flr SF=0, Low Qual Fin SF=0, Gr Liv Area=1656, Bsmt Full Bath=1, Bsmt Half Bath=0, Full Bath=1, Half Bath=0, Bedroom AbvGr=3, Kitchen AbvGr=1, Kitchen Qual='TA', TotRms A

In [21]:
# Checking schema
df_housing.printSchema()

root
 |-- Order: integer (nullable = true)
 |-- PID: integer (nullable = true)
 |-- MS SubClass: integer (nullable = true)
 |-- MS Zoning: string (nullable = true)
 |-- Lot Frontage: integer (nullable = true)
 |-- Lot Area: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- Lot Shape: string (nullable = true)
 |-- Land Contour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- Lot Config: string (nullable = true)
 |-- Land Slope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition 1: string (nullable = true)
 |-- Condition 2: string (nullable = true)
 |-- Bldg Type: string (nullable = true)
 |-- House Style: string (nullable = true)
 |-- Overall Qual: integer (nullable = true)
 |-- Overall Cond: integer (nullable = true)
 |-- Year Built: integer (nullable = true)
 |-- Year Remod/Add: integer (nullable = true)
 |-- Roof Style: string (nullable = true)
 |-- Roof Matl: string (nulla

In [27]:
# Print column names
df_housing.columns

['Order',
 'PID',
 'MS SubClass',
 'MS Zoning',
 'Lot Frontage',
 'Lot Area',
 'Street',
 'Alley',
 'Lot Shape',
 'Land Contour',
 'Utilities',
 'Lot Config',
 'Land Slope',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Mas Vnr Type',
 'Mas Vnr Area',
 'Exter Qual',
 'Exter Cond',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin SF 1',
 'BsmtFin Type 2',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Heating',
 'Heating QC',
 'Central Air',
 'Electrical',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Full Bath',
 'Half Bath',
 'Bedroom AbvGr',
 'Kitchen AbvGr',
 'Kitchen Qual',
 'TotRms AbvGrd',
 'Functional',
 'Fireplaces',
 'Fireplace Qu',
 'Garage Type',
 'Garage Yr Blt',
 'Garage Finish',
 'Garage Cars',
 'Garage 

In [29]:
# Checking data entries for each column
df_housing.select(['longitude', 'housing_median_age', 'total_rooms', 'total_bedroom', '']).describe().show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `longitude` cannot be resolved. Did you mean one of the following? [`Mo Sold`, `Yr Sold`, `Alley`, `Bldg Type`, `Condition 1`].;
'Project ['longitude, 'housing_median_age, 'total_rooms, 'total_bedroom, ']
+- Relation [Order#17,PID#18,MS SubClass#19,MS Zoning#20,Lot Frontage#21,Lot Area#22,Street#23,Alley#24,Lot Shape#25,Land Contour#26,Utilities#27,Lot Config#28,Land Slope#29,Neighborhood#30,Condition 1#31,Condition 2#32,Bldg Type#33,House Style#34,Overall Qual#35,Overall Cond#36,Year Built#37,Year Remod/Add#38,Roof Style#39,Roof Matl#40,... 58 more fields] csv
